In [125]:
!pip install -q fastembed firecrawl-py langchain_community langchain-mistralai crawl4ai[all] playwright crawl4ai langchain-groq langchain-core lxml langchain langchain-qdrant

In [126]:
!playwright install chrome

Failed to install browsers
Error: 
╔═════════════════════════════════════════════════════════════════╗
║ ATTENTION: "chrome" is already installed on the system!         ║
║                                                                 ║
║ "chrome" installation is not hermetic; installing newer version ║
║ requires *removal* of a current installation first.             ║
║                                                                 ║
║ To *uninstall* current version and re-install latest "chrome":  ║
║                                                                 ║
║ - Close all running instances of "chrome", if any               ║
║ - Use "--force" to install browser:                             ║
║                                                                 ║
║     playwright install --force chrome                           ║
║                                                                 ║
║ <3 Playwright Team                                              ║
╚════════════

In [127]:
from langchain_community.document_loaders.firecrawl import FireCrawlLoader

In [128]:
from google.colab import userdata

loader = FireCrawlLoader(
    api_key=userdata.get('FIRECRAWL'),url='https://thegrigorian.medium.com/understanding-vgg-neural-networks-architecture-and-implementation-400d99a9e9ba', mode="scrape"
)
doc = loader.load()

In [129]:
from crawl4ai import AsyncWebCrawler

url1='https://thegrigorian.medium.com/understanding-vgg-neural-networks-architecture-and-implementation-400d99a9e9ba/'

In [130]:
url = 'https://books.toscrape.com/catalogue/scott-pilgrims-precious-little-life-scott-pilgrim-1_987/index.html'
from crawl4ai import AsyncWebCrawler

async def parse_web(url):
    # Create an instance of AsyncWebCrawler
    async with AsyncWebCrawler(verbose=False) as crawler:
        # Run the crawler on a URL
        result = await crawler.arun(url=url)

        # Print the extracted content
        return result

In [131]:
from langchain.schema import Document
docs = []

for i in [url1,url]:
  x = await parse_web(i)
  docs.append(Document(page_content=str(x.markdown_v2),metadata=x.metadata))

[INIT].... → Crawl4AI 0.4.23
[FETCH]... ↓ https://thegrigorian.medium.com/understanding-vgg-... | Status: True | Time: 0.02s
[SCRAPE].. ◆ Processed https://thegrigorian.medium.com/understanding-vgg-... | Time: 965ms
[COMPLETE] ● https://thegrigorian.medium.com/understanding-vgg-... | Status: True | Total: 1.21s
[INIT].... → Crawl4AI 0.4.23
[FETCH]... ↓ https://books.toscrape.com/catalogue/scott-pilgrim... | Status: True | Time: 0.05s
[SCRAPE].. ◆ Processed https://books.toscrape.com/catalogue/scott-pilgrim... | Time: 201ms
[COMPLETE] ● https://books.toscrape.com/catalogue/scott-pilgrim... | Status: True | Total: 0.36s


In [132]:
import os
import time
import warnings
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import Runnable
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
#from langchain_huggingface import HuggingFaceEmbeddings
from rank_bm25 import BM25Okapi

from typing import cast

In [133]:
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["MISTRAL_API_KEY"] =  userdata.get('MISTRAL_API_KEY')

In [134]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(docs)

In [135]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

embeddings = FastEmbedEmbeddings()

In [136]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="fusion-rag27-dec",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="fusion-rag27-dec",
    embedding=embeddings,
)
vector_store.add_documents(doc_splits)

['98e1a35f0b9548e692445119e41fdfa0',
 '3d58433d9055436f98073c14c057becc',
 '968e91b8287b4bdc8301b95da85fe9cd',
 'fa1f10fa63ec4b8f8bff286ab0260813',
 '978660bd2fc3431b84d582aec1f41913',
 '613a4fd89e15486c8bb0dffa5692b50a',
 'aa200e0a7b1543c5b0f084794e0f6bf8',
 'a21d0652265f4034b4240317cfe43855',
 'e4d3def2b30044c3a485069056cbeb56',
 '7f0f6665c1dc41a9b74dd262386ba8f2',
 '0659c2fc90674a1f99f6861c3fa14225',
 '6f7ec43085d64edaa5c6c6446b5c83ff',
 'ae5cbd4a3af24f0f908663e764f4dbde',
 'a6787c6238d94f3c87fd521af6d5ec0c',
 '88871048c35349edb85de19fee5f13d2',
 '6464711d91ec42c7847deeff82deb22a',
 '790632671d9a4cc9a56edb1725ad835c',
 '1e63130875714229bc137047bf0b21d4',
 'efd8817cf3414cb38dd6aa54f0314e93',
 'b25354c2b6494cdcab84bb1dc68a7ff3',
 '4aa57e41ffff4a25ad76aa9264f1efe4',
 '5ec15e362aed47c788413f2b62f3b27b',
 '9bebeafea2a146c88fd3ee07835899a9',
 '241b61f04c6e4c49a627428758bf8d9c',
 '325a1fa78902409684e98eae9af8b09e',
 '9381a097d0d94b538573e88f116507dc',
 'a1c29ea1e6ca481b933e170039b1664c',
 

In [137]:
retriever = vector_store.as_retriever()

In [138]:
retriever.invoke('what is vgg')

[Document(metadata={'title': 'Understanding VGG Neural Networks: Architecture and Implementation | by Anna Alexandra Grigoryan | Medium', 'description': 'VGG (Visual Geometry Group) is a convolutional neural network architecture that was proposed by researchers from the University of Oxford in 2014. It gained popularity and recognition for its…', 'keywords': None, 'author': 'Anna Alexandra Grigoryan', 'og:site_name': 'Medium', 'og:type': 'article', 'og:title': 'Understanding VGG Neural Networks: Architecture and Implementation', 'og:description': 'VGG’s depth-powered brilliance.', 'og:url': 'https://thegrigorian.medium.com/understanding-vgg-neural-networks-architecture-and-implementation-400d99a9e9ba', 'og:image': 'https://miro.medium.com/v2/resize:fit:1024/1*1MDndESIYun5h9LjJZ7k9w.png', 'twitter:app:name:iphone': 'Medium', 'twitter:app:id:iphone': '828256236', 'twitter:site': '@Medium', 'twitter:app:url:iphone': 'medium://p/400d99a9e9ba', 'twitter:image:src': 'https://miro.medium.com/

In [139]:
from langchain_mistralai import ChatMistralAI

model = ChatMistralAI(model="open-mistral-nemo")

In [140]:
model.invoke('what is ai')

AIMessage(content='Artificial Intelligence (AI) is a broad field of computer science dedicated to creating machines that can carry out tasks in a way that we would consider "smart" or "intelligent". Here are some key aspects of AI:\n\n1. **Machine Learning (ML)**: This is a subset of AI that involves training models on data to make predictions or decisions without being explicitly programmed. It\'s how systems like Netflix recommend movies or Amazon suggests products.\n\n2. **Deep Learning (DL)**: This is a subset of ML that uses artificial neural networks with many layers to extract high-level features from raw input. For example, image classification or speech recognition.\n\n3. **Natural Language Processing (NLP)**: This is a subfield of AI that focuses on the interaction between computers and human language. It enables computers to understand, interpret, and generate human language.\n\n4. **Computer Vision**: This is an AI field that focuses on enabling computers to interpret and u

In [141]:
from langchain.prompts import PromptTemplate

query_prompt = PromptTemplate(
    input_variables=["query"],
    template=(
        """You are an expert in modifying queries for better retrieval,\n\n
        Here is one query modify 3 similar queries for better information retrieval and generation\n\n
        Query:{query}\n\n
        Generated query:\n\n"""

    ))

In [142]:
query_chain = query_prompt | model

In [143]:
original_query = query_chain.invoke('what is vgg16 according to the documents?')

In [144]:
import re

queries = re.findall(r'"(.*?)"', original_query.content)

queries

['Provide a brief definition of VGG16 from the documents.',
 'Describe the architectural details of VGG16 as mentioned in the documents.',
 'What are the primary use cases or applications of VGG16, as discussed in the documents?']

In [145]:
from langchain.retrievers import BM25Retriever

retriever1 = BM25Retriever.from_documents(doc_splits)
retriever1.invoke('what is vgg')

[Document(metadata={'title': 'Understanding VGG Neural Networks: Architecture and Implementation | by Anna Alexandra Grigoryan | Medium', 'description': 'VGG (Visual Geometry Group) is a convolutional neural network architecture that was proposed by researchers from the University of Oxford in 2014. It gained popularity and recognition for its…', 'keywords': None, 'author': 'Anna Alexandra Grigoryan', 'og:site_name': 'Medium', 'og:type': 'article', 'og:title': 'Understanding VGG Neural Networks: Architecture and Implementation', 'og:description': 'VGG’s depth-powered brilliance.', 'og:url': 'https://thegrigorian.medium.com/understanding-vgg-neural-networks-architecture-and-implementation-400d99a9e9ba', 'og:image': 'https://miro.medium.com/v2/resize:fit:1024/1*1MDndESIYun5h9LjJZ7k9w.png', 'twitter:app:name:iphone': 'Medium', 'twitter:app:id:iphone': '828256236', 'twitter:site': '@Medium', 'twitter:app:url:iphone': 'medium://p/400d99a9e9ba', 'twitter:image:src': 'https://miro.medium.com/

In [146]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(retrievers=[retriever1, retriever], weights=[0.5, 0.5])
retrievers = [retriever,retriever1,ensemble_retriever]

In [147]:
query = 'what is vgg'
def retrieve_docs(retriever,query):
  return retriever.invoke(query)
r_docs = []
for query in queries:
  q_docs = []
  for retriever in retrievers:
    docs = retrieve_docs(retriever,query)
    q_docs.append(docs)
  q_docs = [x for i in q_docs for x in i]
  r_docs.append(q_docs)
r_docs = [x for i in r_docs for x in i]

In [148]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def reranked_docs(docs):
  docs_content = [doc.page_content for doc in docs]
  vectorizer = TfidfVectorizer()
  doc_vectors = vectorizer.fit_transform(docs_content)
  query_vector = vectorizer.transform([query])
  similarity = cosine_similarity(query_vector,doc_vectors).flatten()
  scored_docs = sorted(zip(docs_content, similarity), key=lambda x: x[1], reverse=True)
  return scored_docs
ranked_docs = reranked_docs(r_docs)[:3]

In [149]:
llm = ChatMistralAI(model="open-mistral-nemo")
rag_prompt = hub.pull("rlm/rag-prompt")
rag_chain = rag_prompt | llm

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [150]:
rag_chain.invoke({'context':ranked_docs,'question':query})

AIMessage(content="VGG16 is primarily used for image classification tasks due to its depth and ability to capture intricate features. It's also used in object detection and recognition, especially in cluttered scenes.", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 893, 'total_tokens': 931, 'completion_tokens': 38}, 'model': 'open-mistral-nemo', 'finish_reason': 'stop'}, id='run-7291ec59-b76d-43c2-bbdb-fafb30fa501a-0', usage_metadata={'input_tokens': 893, 'output_tokens': 38, 'total_tokens': 931})